In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\admin\appdata\local\temp\pip-req-build-t3z9s497
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit dcf6bde8d64e1a747673d24dde30dce195bc3cf1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\admin\appdata\local\temp\pip-install-hxb94b2s\elegantrl_444aeeae767b4f04bc6d886ce1a92eb5
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 37aac1f592e1add9f9fd37ae8db1094656009b76
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-t3z9s497'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\Admin\AppData\Local\Temp\pip-install-hxb94b2s\elegantrl_444aeeae767b4f04bc6d886ce1a92eb5'


In [4]:
import pandas as pd
import numpy as np
import datetime

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

In [6]:
## Có thể gọi các class chi tiết trong thư viện thay vì sử dụng cấp cao nhất là Vnstock
from vnstock import Listing, Quote, Company, Finance, Trading, Screener 

In [17]:
quote = Quote(symbol='ACB', source='VCI')
df = quote.history(start='2024-01-01', end='2025-03-05', interval='1D')
df

,time,open,high,low,close,volume
0,2024-01-02,16.81,17.37,16.81,17.16,13896933
1,2024-01-03,17.19,17.55,17.02,17.55,9817807
2,2024-01-04,17.69,18.00,17.62,17.76,23605373
3,2024-01-05,17.76,17.86,17.58,17.86,9282598
4,2024-01-08,18.04,18.07,17.69,17.79,12398885
...,...,...,...,...,...,...
285,2025-02-27,21.72,21.81,21.60,21.81,5661613
286,2025-02-28,21.72,21.89,21.56,21.72,5475462
287,2025-03-03,21.72,21.76,21.56,21.72,5603548
288,2025-03-04,21.64,21.89,21.56,21.85,9964154


In [18]:
# ...existing code...
# debug: xem các cột hiện tại
print("columns:", df.columns.tolist())

# đảm bảo có cột 'tic' (tạo từ symbol/ticker hoặc từ biến quote nếu cần)
if 'tic' not in df.columns:
    if 'symbol' in df.columns:
        df['tic'] = df['symbol'].astype(str).str.lower()
    elif 'ticker' in df.columns:
        df['tic'] = df['ticker'].astype(str).str.lower()
    else:
        try:
            df['tic'] = quote.symbol.lower()   # nếu bạn có biến quote
        except Exception:
            df['tic'] = 'unknown'

# rename time -> date nếu cần và ép kiểu datetime
if 'time' in df.columns and 'date' not in df.columns:
    df = df.rename(columns={'time': 'date'})
df['date'] = pd.to_datetime(df['date'])

# map các tên OHLCV thay thế
cols_map = {}
for target in ['open','high','low','close','volume']:
    for alt in [target.upper(), target.capitalize(), target + '_price', target + '_PRICE']:
        if alt in df.columns and target not in df.columns:
            cols_map[alt] = target
if cols_map:
    df = df.rename(columns=cols_map)

# tạo cột day và sắp xếp; nếu 'tic' vẫn không có thì chỉ sort theo date
df['day'] = df['date'].dt.dayofweek
if 'tic' in df.columns:
    df = df.sort_values(['date','tic']).reset_index(drop=True)
else:
    df = df.sort_values('date').reset_index(drop=True)

desired = ['date','open','high','low','close','volume','tic','day']
df = df[[c for c in desired if c in df.columns]]

df.head()
# ...existing code...

columns: ['time', 'open', 'high', 'low', 'close', 'volume']


,date,open,high,low,close,volume,tic,day
0,2024-01-02,16.81,17.37,16.81,17.16,13896933,acb,1
1,2024-01-03,17.19,17.55,17.02,17.55,9817807,acb,2
2,2024-01-04,17.69,18.00,17.62,17.76,23605373,acb,3
3,2024-01-05,17.76,17.86,17.58,17.86,9282598,acb,4
4,2024-01-08,18.04,18.07,17.69,17.79,12398885,acb,0


In [19]:
tech_indicator_list = ["macd", "rsi", "cci", "dx", "roc", "atr"]

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=tech_indicator_list,
                     use_turbulence=False,
                     user_defined_feature=False)

# ...existing code...
# DEBUG + FIX trước khi FeatureEngineer.preprocess_data
print("INDICATORS from finrl.config:", INDICATORS)

# Ensure core columns exist

required = ['date','tic','close','open','high','low','volume']
print("columns before normalize:", df.columns.tolist())
for c in ['date','tic','close']:
    if c not in df.columns:
        raise RuntimeError(f"Missing required column: {c} - check earlier normalization")

# convert date, ensure tic lowercase
df['date'] = pd.to_datetime(df['date'])
df['tic'] = df['tic'].astype(str).str.lower()

# drop ticker groups that are too short for indicators (avoid indicator not created)
min_needed = 60   # choose >= max window used by indicators (e.g. 30/60)
grp_sizes = df.groupby('tic').size()
print("group sizes (sample):", grp_sizes.head().to_dict())
short_tics = grp_sizes[grp_sizes < min_needed].index.tolist()
if short_tics:
    print(f"Dropping {len(short_tics)} tickers with <{min_needed} rows:", short_tics[:10])
    df = df[~df['tic'].isin(short_tics)].copy()
    df = df.reset_index(drop=True)

# Use INDICATORS or pick a safe subset — remove 'roc' if it's causing trouble
tech_indicator_list = ["macd", "rsi", "cci", "dx", "atr"]  # removed 'roc'
print("tech_indicator_list used:", tech_indicator_list)

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=tech_indicator_list,
                     use_turbulence=False,
                     user_defined_feature=False)

# Try add_technical_indicator and inspect result
try:
    tmp = fe.add_technical_indicator(df.copy())
    new_cols = [c for c in tmp.columns if c not in df.columns]
    print("add_technical_indicator succeeded. New columns count:", len(new_cols))
    print("New columns sample:", new_cols[:30])
except Exception as e:
    print("add_technical_indicator ERROR:", type(e).__name__, e)
    # inspect small sample per tic to find why indicators failed
    for t, g in df.groupby('tic'):
        print("tic", t, "rows:", len(g))
        print(g[['date','close']].head(5))
        break
    raise

# proceed to full preprocess if OK
processed_df = fe.preprocess_data(df)
print("processed_df shape:", processed_df.shape)
# ...existing code...

INDICATORS from finrl.config: ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
columns before normalize: ['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day']
group sizes (sample): {'acb': 290}
tech_indicator_list used: ['macd', 'rsi', 'cci', 'dx', 'atr']
add_technical_indicator succeeded. New columns count: 5
New columns sample: ['macd', 'rsi', 'cci', 'dx', 'atr']
Successfully added technical indicators
processed_df shape: (290, 13)


In [20]:
processed_df

,date,open,high,low,close,volume,tic,day,macd,rsi,cci,dx,atr
0,2024-01-02,16.81,17.37,16.81,17.16,13896933,acb,1,0.000000,100.000000,66.666667,100.000000,0.560000
1,2024-01-03,17.19,17.55,17.02,17.55,9817807,acb,2,0.008750,100.000000,66.666667,100.000000,0.544444
2,2024-01-04,17.69,18.00,17.62,17.76,23605373,acb,3,0.017661,100.000000,100.000000,100.000000,0.510603
3,2024-01-05,17.76,17.86,17.58,17.86,9282598,acb,4,0.024892,100.000000,63.354037,85.845718,0.446395
4,2024-01-08,18.04,18.07,17.69,17.79,12398885,acb,0,0.025512,89.332267,67.129630,90.092974,0.431079
...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,2025-02-27,21.72,21.81,21.60,21.81,5661613,acb,3,0.192227,63.746487,60.728574,31.023609,0.230342
286,2025-02-28,21.72,21.89,21.56,21.72,5475462,acb,4,0.184323,59.770220,42.804200,39.991613,0.237461
287,2025-03-03,21.72,21.76,21.56,21.72,5603548,acb,0,0.176031,59.770220,12.740741,39.991613,0.234785
288,2025-03-04,21.64,21.89,21.56,21.85,9964154,acb,1,0.177898,63.576266,58.333333,53.165416,0.241586
